In [3]:
import json
import pennylane as qml
import pennylane.numpy as np

def potential_energy_surface(symbols, bond_lengths):
    """Calculates the molecular energy over various bond lengths (AKA the 
    potential energy surface) using the Hartree Fock method.
    
    Args:
        symbols (list(string)): 
            A list of atomic symbols that comprise the diatomic molecule of interest.
        bond_lengths (numpy.tensor): Bond lengths to calculate the energy over.

        
    Returns:
        hf_energies (numpy.tensor): 
            The Hartree Fock energies at every bond length value.
    """

    # Put your code here #
    hf_energies = []

    alpha = bond_lengths 
    args = [alpha]
    for length in bond_lengths:
        geometry = np.array([[0.0, 0.0, 0.0], [length, 0.0, 0.0]], requires_grad = False)
        mol = qml.qchem.Molecule(symbols,geometry)
        hf_energies.append(qml.qchem.hf_energy(mol)())
    
    return np.array(hf_energies)


def ground_energy(hf_energies):
    """Finds the minimum energy of a molecule given its potential energy surface.
    
    Args: 
        hf_energies (numpy.tensor): 

    Returns:
        (float): The minumum energy in units of hartrees.
    """

    ind = np.argmin(hf_energies)
    return hf_energies[ind]

def reaction():
    """Calculates the energy of the reactants, the activation energy, and the energy of 
    the products in that order.

    Returns:
        (numpy.tensor): [E_reactants, E_activation, E_products]
    """
    molecules = {
        "H2": 
            {"symbols": ["H", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(0.5, 9.3, 0.3)}, 
        "Li2": 
            {"symbols": ["Li", "Li"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(3.5, 8.3, 0.3)}, 
        "LiH": 
            {"symbols": ["Li", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(2.0, 6.6, 0.3)}
    }


    for molecule in molecules.keys():
        # Calculate the UCCSD energy at each bond length
        uccsd_energies = potential_energy_surface(molecules[molecule]["symbols"], molecules[molecule]["bond lengths"])

        # Find the minimum energy for reactants and dissociation
        E0 = ground_energy(uccsd_energies)
        Edissoc = np.abs(E0 - uccsd_energies[-1])

        # Update molecule properties
        molecules[molecule]["E0"] = E0
        molecules[molecule]["E_dissociation"] = Edissoc

    # Calculate the activation energy and energies of the reactants and products
    E_reactants = molecules["H2"]["E0"] + molecules["Li2"]["E0"]
    E_activation = E_reactants + molecules["H2"]["E_dissociation"] + molecules["Li2"]["E_dissociation"]
    E_products = 2 * molecules["LiH"]["E0"]

    return np.array([E_reactants, E_activation, E_products])


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    output = reaction().tolist()
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    assert np.allclose(solution_output, expected_output, rtol=1e-3)

In [4]:
molecules = {
        "H2": 
            {"symbols": ["H", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(0.5, 9.3, 0.3)}, 
        "Li2": 
            {"symbols": ["Li", "Li"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(3.5, 8.3, 0.3)}, 
        "LiH": 
            {"symbols": ["Li", "H"], "E0": 0, "E_dissociation": 0, "bond lengths": np.arange(2.0, 6.6, 0.3)}}

In [5]:
list(molecules.items())[0][1]

{'symbols': ['H', 'H'],
 'E0': 0,
 'E_dissociation': 0,
 'bond lengths': tensor([0.5, 0.8, 1.1, 1.4, 1.7, 2. , 2.3, 2.6, 2.9, 3.2, 3.5, 3.8, 4.1,
         4.4, 4.7, 5. , 5.3, 5.6, 5.9, 6.2, 6.5, 6.8, 7.1, 7.4, 7.7, 8. ,
         8.3, 8.6, 8.9, 9.2], requires_grad=True)}

In [6]:
symbols = list(molecules.items())[0][1]['symbols']
alpha = list(molecules.items())[0][1]['bond lengths']
bond_lengths = alpha
args = [alpha]
hf_energies = []
for length in bond_lengths:
    geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, length]], requires_grad = False)
    mol = qml.qchem.Molecule(symbols,geometry)
    hf_energies.append(qml.qchem.hf_energy(mol)(*args))
print(np.array(hf_energies))

[-0.40332644 -0.94730793 -1.0945641  -1.11671433 -1.09200585 -1.0491709
 -0.99964323 -0.94904315 -0.90060359 -0.85609692 -0.81634416 -0.78156784
 -0.75161246 -0.72609682 -0.70453368 -0.68641593 -0.67126398 -0.65864203
 -0.64815813 -0.63946079 -0.63223745 -0.626215   -0.62116033 -0.61687936
 -0.6132142  -0.6100389  -0.6072544  -0.6047836  -0.60256683 -0.60055796]


In [7]:
uccsd_energies = np.array(hf_energies)

In [8]:
E0 = ground_energy(uccsd_energies)
Edissoc = np.abs(E0 - uccsd_energies[-1])

In [9]:
molecules["H2"]["E0"] = E0
molecules["H2"]["E_dissociation"] = Edissoc

In [10]:
for molecule in molecules.keys():
        # Calculate the UCCSD energy at each bond length
        uccsd_energies = potential_energy_surface(molecules[molecule]["symbols"], molecules[molecule]["bond lengths"])

        # Find the minimum energy for reactants and dissociation
        E0 = ground_energy(uccsd_energies)
        Edissoc = np.abs(E0 - uccsd_energies[-1])

        # Update molecule properties
        molecules[molecule]["E0"] = E0
        molecules[molecule]["E_dissociation"] = Edissoc

    # Calculate the activation energy and energies of the reactants and products
E_reactants = molecules["H2"]["E0"] + molecules["Li2"]["E0"]
E_activation = E_reactants + molecules["H2"]["E_dissociation"] + molecules["Li2"]["E_dissociation"]
E_products = 2 * molecules["LiH"]["E0"]

np.array([E_reactants, E_activation, E_products])

tensor([-15.7553572 , -15.10600089, -15.72653446], requires_grad=True)